✅ Parte 1 — Carregar a Silver deflacionada

In [0]:
from pyspark.sql import functions as F

df = spark.table("workspace.mvp.silver_dca_deflacionada")

display(df.limit(10))
df.printSchema()


✅ Parte 2 — Criar métricas (nominais + reais)

Vamos criar todas as métricas que você precisa para responder seus questionamentos:
investimento / despesa total (nominal e real)
investimento / receita total (nominal e real)
investimento real per capita
participação de despesa corrente na despesa total (real)
algumas proteções contra divisão por zero

In [0]:
def safe_div(num_col, den_col):
    return F.when((den_col.isNull()) | (den_col == 0), None).otherwise(num_col / den_col)

df_gold = (
    df
    # Métricas NOMINAIS
    .withColumn("Investimento_pct_Despesa", safe_div(F.col("Investimento"), F.col("Despesa_Total")))
    .withColumn("Investimento_pct_Receita", safe_div(F.col("Investimento"), F.col("Receita_Total")))

    # Métricas REAIS (deflacionadas)
    .withColumn("Investimento_Real_pct_Despesa_Real", safe_div(F.col("Investimento_Real"), F.col("Despesa_Total_Real")))
    .withColumn("Investimento_Real_pct_Receita_Real", safe_div(F.col("Investimento_Real"), F.col("Receita_Total_Real")))

    # Per capita (real)
    .withColumn("Investimento_Real_per_capita", safe_div(F.col("Investimento_Real"), F.col("Populacao")))

    # Prioridade de despesa corrente (real)
    .withColumn("Despesa_Corrente_Real_pct_Despesa_Real", safe_div(F.col("Despesa_Corrente_Real"), F.col("Despesa_Total_Real")))
)

display(df_gold.limit(20))


✅ Parte 3 — Validar rapidamente (recomendado)

In [0]:
# Checar se tem anos/ufs faltando ou valores nulos demais
display(
    df_gold.select(
        F.count("*").alias("linhas"),
        F.count(F.when(F.col("Investimento_Real").isNull(), 1)).alias("investimento_real_null"),
        F.count(F.when(F.col("Despesa_Total_Real").isNull(), 1)).alias("despesa_total_real_null"),
        F.count(F.when(F.col("Receita_Total_Real").isNull(), 1)).alias("receita_total_real_null")
    )
)


✅ Parte 4 — Salvar a tabela GOLD

In [0]:
df_gold.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.mvp.gold_dca")

print("✅ Tabela GOLD criada: workspace.mvp.gold_dca")
